In [1]:
%pylab inline
import pandas as pd
from pyro.contrib.brm import defm

Populating the interactive namespace from numpy and matplotlib


### Design space coding

How would we replicate the coding used in the friendliness example using brmp?

#### Recap

Simuli in the friendliness example are left/right pairs of images. Each image has two feature dimensions -- mouth and eye brows. Each dimension can take on one of three values. Here I'll consider a simpler setting with only a single feature dimension. Now an entire image can be coded as a one-hot 3 vector. e.g. `[0,1,0]`. A *pair* of images can be coded as the difference between two such vectors. This approach leads to a stimuli coding scheme that can take on 7 possible values, something like:

```
[ 0,  0,  0]
[ 0,  1, -1]
[ 1,  0, -1]
[ 0, -1,  1]
[ 1, -1,  0]
[-1,  0,  1]
[-1,  1,  0]
```

#### Over in brmp...

The default mode of operation for brms/brmp is for the system to handle coding. The obvious way to include a stimulus that can take on 7 different values in the model as follows:

In [2]:
# Example data:
df = pd.DataFrame(dict(
    y=[0., 0., 0., 0., 0., 0., 0.],
    stimulus=pd.Categorical(['a', 'b', 'c', 'd', 'e', 'f', 'g']),
))
model = defm('y ~ stimulus', df)
model

Population
----------------------------------------
Coef Priors:
stimulus[a]     | Cauchy(loc=0.0, scale=1.0)
stimulus[b]     | Cauchy(loc=0.0, scale=1.0)
stimulus[c]     | Cauchy(loc=0.0, scale=1.0)
stimulus[d]     | Cauchy(loc=0.0, scale=1.0)
stimulus[e]     | Cauchy(loc=0.0, scale=1.0)
stimulus[f]     | Cauchy(loc=0.0, scale=1.0)
stimulus[g]     | Cauchy(loc=0.0, scale=1.0)
Response
----------------------------------------
Family: Normal()
Link:
  Parameter: mu
  Function:  identity
Priors:
sigma           | HalfCauchy(scale=3.0)

From this model specification brmp picks a suitable (in some sense) approch to coding the data. In this case, each possible value of `stimulus` is encoded as a one-hot vector of length 7. Here's our data frame again, followed by the design matrix generated by brmp:

In [3]:
df

,y,stimulus
0,0.0,a
1,0.0,b
2,0.0,c
3,0.0,d
4,0.0,e
5,0.0,f
6,0.0,g


In [4]:
model.data['X']

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.]])

#### Possible interface for custom coding

This is often OK, but it's not what we want for the friendliness example. Instead we want to use the coding described earlier. A direct way to do this would be to have the system accept as an argument a `7xM` matrix describing the vector of length `M` that should be used to code each value the stimulus can take. Something like:

In [5]:
codes = [
    [ 0,  0,  0],
    [ 0,  1, -1],
    [ 1,  0, -1],
    [ 0, -1,  1],
    [ 1, -1,  0],
    [-1,  0,  1],
    [-1,  1,  0],
]

# Proposed interface (not yet implemented...)
#defm('y ~ simulus', df=df, contrasts={'stimulus': codes})

I think the linear model folk would call such a thing a contrast matrix. (Though I think that concept is much broader than what I'm proposing we add here.)

Note that implementing this isn't quite as trivial as it may seem, since we need to code interactions (e.g. `y ~ stimulus:other`) appropriately.

Also, note that the suggested interface assumes that the same coding will be used for each occurance of the factor/column in the formula. That is, it doesn't allow the two occurances of `stimulus` in e.g. `y ~ stimulus + (stimulus | id)` to use different codings.



#### Questions

* Does this seem like a reasonable approach?